<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.1 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 4.1 MB 35.5 MB/s 
     |████████████████████████████████| 110 kB 62.6 MB/s 
     |████████████████████████████████| 128 kB 61.7 MB/s 
     |████████████████████████████████| 112 kB 80.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Looking in indexes: https://pypi.org/simple, https

In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA',
       'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY',
       'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    54
1    10
Name: InPortfolio, dtype: int64

In [6]:
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0.0    7
1.0    3
Name: CCIFlag, dtype: int64

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [8]:
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [9]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI
Date,,,,,,,,,,
2022-12-27 00:00:00+05:30,18132.0,^NSEI,5.38,0.50,0.52,18812.0,15294.0,80.68,19.32,45.0
2022-12-28 00:00:00+05:30,18122.0,^NSEI,5.27,0.51,0.50,18812.0,15294.0,80.40,19.60,45.0
2022-12-29 00:00:00+05:30,18191.0,^NSEI,5.62,0.52,0.45,18812.0,15294.0,82.35,17.65,47.0
2022-12-30 00:00:00+05:30,18105.0,^NSEI,5.07,0.50,0.44,18812.0,15294.0,79.91,20.09,44.0
2023-01-02 00:00:00+05:30,18197.0,^NSEI,5.57,0.49,0.40,18812.0,15294.0,82.53,17.47,48.0


In [11]:
# MACD Uptrend
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)

In [12]:
# portfolio stocks
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
1,2023-01-02 00:00:00+05:30,452.0,ICICIPRULI.NS,-12.33,0.00,-1.27,615.0,440.0,7.09,92.91,44.0,640.0,60.9,76.00,42.0
3,2023-01-02 00:00:00+05:30,695.0,MOTILALOFS.NS,-9.66,-0.91,-0.15,960.0,654.0,13.32,86.68,54.0,930.0,19.7,9.54,34.0
4,2023-01-02 00:00:00+05:30,1240.0,SBILIFE.NS,3.86,0.75,0.24,1332.0,1023.0,70.24,29.76,47.0,1607.0,62.1,1000.00,30.0
7,2023-01-02 00:00:00+05:30,395.0,TATAMOTORS.NS,-7.76,-0.23,-0.24,525.0,372.0,14.90,85.10,42.0,495.0,19.2,1000.00,25.0
11,2023-01-02 00:00:00+05:30,3912.0,NAUKRI.NS,-3.63,-0.62,0.23,5667.0,3363.0,23.81,76.19,43.0,4677.0,78.8,17.00,20.0
13,2023-01-02 00:00:00+05:30,570.0,HDFCLIFE.NS,2.76,0.36,0.71,672.0,502.0,40.18,59.82,50.0,681.0,89.3,83.20,19.0
23,2023-01-02 00:00:00+05:30,1524.0,INFY.NS,-0.33,-0.78,0.26,1898.0,1351.0,31.63,68.37,46.0,1715.0,23.0,28.00,13.0
24,2023-01-02 00:00:00+05:30,1010.0,TECHM.NS,-7.89,-1.82,0.10,1734.0,935.0,9.32,90.68,42.0,1139.0,18.7,18.60,13.0
26,2023-01-02 00:00:00+05:30,1092.0,MUTHOOTFIN.NS,-1.47,-1.26,0.37,1517.0,956.0,24.19,75.81,53.0,1217.0,12.5,11.70,11.0
48,2023-01-02 00:00:00+05:30,3373.0,DIVISLAB.NS,-10.09,-1.01,-0.53,4616.0,3266.0,7.90,92.10,46.0,3560.0,45.2,29.90,6.0


In [13]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 4


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
2,2023-01-02 00:00:00+05:30,577.0,BERGEPAINT.NS,-8.37,-0.63,-0.50,787.0,558.0,8.43,91.57,35.0,782.0,72.9,59.3,36.0
5,2023-01-02 00:00:00+05:30,505.0,JUBLFOOD.NS,-9.33,-0.18,-1.63,785.0,463.0,13.06,86.94,34.0,643.0,87.9,70.4,27.0
10,2023-01-02 00:00:00+05:30,1101.0,HAVELLS.NS,-10.37,0.08,-1.09,1400.0,1068.0,9.82,90.18,27.0,1328.0,63.7,63.4,21.0
14,2023-01-02 00:00:00+05:30,2566.0,TITAN.NS,5.28,0.12,-0.31,2770.0,1929.0,75.71,24.29,51.0,2967.0,78.0,71.9,16.0


In [14]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%']*100/tmp_df['Upside%'].sum()

1     19.718310
3     15.962441
4     14.084507
7     11.737089
11     9.389671
13     8.920188
23     6.103286
24     6.103286
26     5.164319
48     2.816901
Name: Upside%, dtype: float64

In [15]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 11


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
0,2023-01-02 00:00:00+05:30,435.0,GODREJIND.NS,-4.17,-0.88,0.23,644.0,405.0,12.57,87.43,44.0,638.0,39.5,18.9,47.0
2,2023-01-02 00:00:00+05:30,577.0,BERGEPAINT.NS,-8.37,-0.63,-0.50,787.0,558.0,8.43,91.57,35.0,782.0,72.9,59.3,36.0
5,2023-01-02 00:00:00+05:30,505.0,JUBLFOOD.NS,-9.33,-0.18,-1.63,785.0,463.0,13.06,86.94,34.0,643.0,87.9,70.4,27.0
6,2023-01-02 00:00:00+05:30,243.0,ZEEL.NS,-3.15,0.00,-1.17,325.0,206.0,31.18,68.82,40.0,303.0,24.6,28.8,25.0
8,2023-01-02 00:00:00+05:30,6554.0,BAJFINANCE.NS,-1.88,-0.04,-1.24,7824.0,5265.0,50.36,49.64,45.0,8015.0,50.8,40.0,22.0
9,2023-01-02 00:00:00+05:30,902.0,ICICIBANK.NS,10.73,1.23,0.22,953.0,650.0,83.32,16.68,47.0,1094.0,24.2,21.2,21.0
10,2023-01-02 00:00:00+05:30,1101.0,HAVELLS.NS,-10.37,0.08,-1.09,1400.0,1068.0,9.82,90.18,27.0,1328.0,63.7,63.4,21.0
12,2023-01-02 00:00:00+05:30,3229.0,EICHERMOT.NS,5.11,1.43,-0.99,3850.0,2241.0,61.40,38.60,43.0,3884.0,40.1,37.8,20.0
14,2023-01-02 00:00:00+05:30,2566.0,TITAN.NS,5.28,0.12,-0.31,2770.0,1929.0,75.71,24.29,51.0,2967.0,78.0,71.9,16.0
15,2023-01-02 00:00:00+05:30,612.0,SBIN.NS,16.39,1.52,1.00,626.0,434.0,92.81,7.19,56.0,710.0,13.2,13.3,16.0


In [16]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 85]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 6


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
37,2023-01-02 00:00:00+05:30,393.0,WIPRO.NS,-10.87,-2.27,0.25,714.0,375.0,5.32,94.68,50.0,425.0,19.0,19.1,8.0
2,2023-01-02 00:00:00+05:30,577.0,BERGEPAINT.NS,-8.37,-0.63,-0.50,787.0,558.0,8.43,91.57,35.0,782.0,72.9,59.3,36.0
10,2023-01-02 00:00:00+05:30,1101.0,HAVELLS.NS,-10.37,0.08,-1.09,1400.0,1068.0,9.82,90.18,27.0,1328.0,63.7,63.4,21.0
0,2023-01-02 00:00:00+05:30,435.0,GODREJIND.NS,-4.17,-0.88,0.23,644.0,405.0,12.57,87.43,44.0,638.0,39.5,18.9,47.0
5,2023-01-02 00:00:00+05:30,505.0,JUBLFOOD.NS,-9.33,-0.18,-1.63,785.0,463.0,13.06,86.94,34.0,643.0,87.9,70.4,27.0
38,2023-01-02 00:00:00+05:30,4322.0,LTIM.NS,-7.65,-1.88,-0.70,7506.0,3796.0,14.18,85.82,40.0,4675.0,26.6,50.0,8.0


In [17]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] < 10]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 9


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
55,2023-01-02 00:00:00+05:30,119.0,TATASTEEL.NS,37.07,3.45,1.85,119.0,44.0,100.33,-0.33,66.0,118.0,6.0,5.07,-1.0
60,2023-01-02 00:00:00+05:30,336.0,CANBK.NS,39.48,2.07,2.57,336.0,175.0,100.09,-0.09,62.0,322.0,8.8,6.50,-4.0
34,2023-01-02 00:00:00+05:30,138.0,FEDERALBNK.NS,25.18,1.82,0.00,139.0,82.0,97.72,2.28,56.0,152.0,11.0,11.70,10.0
61,2023-01-02 00:00:00+05:30,296.0,INDIANB.NS,51.67,4.10,2.94,300.0,127.0,97.54,2.46,58.0,284.0,7.4,8.48,-4.0
25,2023-01-02 00:00:00+05:30,942.0,AXISBANK.NS,22.60,1.56,1.33,950.0,625.0,97.42,2.58,61.0,1060.0,31.3,15.80,13.0
56,2023-01-02 00:00:00+05:30,186.0,BANKBARODA.NS,45.51,3.91,2.96,193.0,81.0,93.97,6.03,62.0,183.0,13.5,9.80,-2.0
15,2023-01-02 00:00:00+05:30,612.0,SBIN.NS,16.39,1.52,1.00,626.0,434.0,92.81,7.19,56.0,710.0,13.2,13.30,16.0
19,2023-01-02 00:00:00+05:30,1226.0,INDUSINDBK.NS,18.60,1.55,0.17,1264.0,771.0,92.36,7.64,56.0,1393.0,22.8,15.60,14.0
32,2023-01-02 00:00:00+05:30,1629.0,HDFCBANK.NS,12.02,0.89,1.13,1662.0,1281.0,91.26,8.74,54.0,1813.0,26.0,21.80,11.0


In [18]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 6


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
52,2023-01-02 00:00:00+05:30,3261.0,TCS.NS,-1.05,-0.42,0.43,3965.0,2974.0,29.01,70.99,47.0,3385.0,28.1,30.1,4.0
39,2023-01-02 00:00:00+05:30,1039.0,HCLTECH.NS,3.22,-0.40,0.37,1291.0,865.0,40.95,59.05,44.0,1117.0,16.3,20.4,8.0
50,2023-01-02 00:00:00+05:30,4073.0,DMART.NS,0.89,-0.20,-0.27,4731.0,3231.0,56.12,43.88,51.0,4282.0,138.7,114.0,5.0
36,2023-01-02 00:00:00+05:30,212.0,TATAPOWER.NS,-7.08,-0.44,-0.45,287.0,195.0,18.32,81.68,46.0,232.0,20.4,26.6,9.0
8,2023-01-02 00:00:00+05:30,6554.0,BAJFINANCE.NS,-1.88,-0.04,-1.24,7824.0,5265.0,50.36,49.64,45.0,8015.0,50.8,40.0,22.0
5,2023-01-02 00:00:00+05:30,505.0,JUBLFOOD.NS,-9.33,-0.18,-1.63,785.0,463.0,13.06,86.94,34.0,643.0,87.9,70.4,27.0


In [19]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 8


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
0,2023-01-02 00:00:00+05:30,435.0,GODREJIND.NS,-4.17,-0.88,0.23,644.0,405.0,12.57,87.43,44.0,638.0,39.5,18.90,47.0
20,2023-01-02 00:00:00+05:30,83.0,IBREALEST.NS,4.06,-2.50,1.22,174.0,60.0,20.39,79.61,53.0,95.0,11.3,1000.00,14.0
37,2023-01-02 00:00:00+05:30,393.0,WIPRO.NS,-10.87,-2.27,0.25,714.0,375.0,5.32,94.68,50.0,425.0,19.0,19.10,8.0
39,2023-01-02 00:00:00+05:30,1039.0,HCLTECH.NS,3.22,-0.40,0.37,1291.0,865.0,40.95,59.05,44.0,1117.0,16.3,20.40,8.0
44,2023-01-02 00:00:00+05:30,84.0,NATIONALUM.NS,-0.36,-2.38,1.32,128.0,67.0,27.38,72.62,66.0,90.0,10.3,6.05,7.0
49,2023-01-02 00:00:00+05:30,487.0,HINDALCO.NS,12.48,-1.39,1.81,626.0,314.0,55.46,44.54,63.0,514.0,10.4,7.90,6.0
52,2023-01-02 00:00:00+05:30,3261.0,TCS.NS,-1.05,-0.42,0.43,3965.0,2974.0,29.01,70.99,47.0,3385.0,28.1,30.10,4.0
59,2023-01-02 00:00:00+05:30,89.0,SAIL.NS,9.81,-1.23,1.20,108.0,63.0,57.67,42.33,64.0,85.0,6.9,7.74,-4.0


In [20]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 7


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
60,2023-01-02 00:00:00+05:30,336.0,CANBK.NS,39.48,2.07,2.57,336.0,175.0,100.09,-0.09,62.0,322.0,8.8,6.5,-4.0
41,2023-01-02 00:00:00+05:30,2043.0,INDIGO.NS,10.54,0.65,1.32,2265.0,1549.0,68.96,31.04,59.0,2176.0,30.7,1000.0,7.0
28,2023-01-02 00:00:00+05:30,2652.0,HDFC.NS,11.68,0.97,1.19,2729.0,2053.0,88.67,11.33,50.0,2940.0,23.0,20.1,11.0
32,2023-01-02 00:00:00+05:30,1629.0,HDFCBANK.NS,12.02,0.89,1.13,1662.0,1281.0,91.26,8.74,54.0,1813.0,26.0,21.8,11.0
46,2023-01-02 00:00:00+05:30,7018.0,ULTRACEMCO.NS,9.64,0.77,1.04,7825.0,5148.0,69.85,30.15,50.0,7420.0,33.7,30.4,6.0
31,2023-01-02 00:00:00+05:30,562.0,DABUR.NS,2.85,0.37,0.71,604.0,480.0,65.77,34.23,41.0,625.0,53.6,55.7,11.0
54,2023-01-02 00:00:00+05:30,870.0,MCDOWELL-N.NS,3.17,0.36,0.56,947.0,732.0,64.05,35.95,36.0,885.0,74.5,56.4,2.0


In [21]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
49,2023-01-02 00:00:00+05:30,487.0,HINDALCO.NS,12.48,-1.39,1.81,626.0,314.0,55.46,44.54,63.0,514.0,10.4,7.90,6.0
44,2023-01-02 00:00:00+05:30,84.0,NATIONALUM.NS,-0.36,-2.38,1.32,128.0,67.0,27.38,72.62,66.0,90.0,10.3,6.05,7.0
20,2023-01-02 00:00:00+05:30,83.0,IBREALEST.NS,4.06,-2.50,1.22,174.0,60.0,20.39,79.61,53.0,95.0,11.3,1000.00,14.0
59,2023-01-02 00:00:00+05:30,89.0,SAIL.NS,9.81,-1.23,1.20,108.0,63.0,57.67,42.33,64.0,85.0,6.9,7.74,-4.0
52,2023-01-02 00:00:00+05:30,3261.0,TCS.NS,-1.05,-0.42,0.43,3965.0,2974.0,29.01,70.99,47.0,3385.0,28.1,30.10,4.0
39,2023-01-02 00:00:00+05:30,1039.0,HCLTECH.NS,3.22,-0.40,0.37,1291.0,865.0,40.95,59.05,44.0,1117.0,16.3,20.40,8.0
37,2023-01-02 00:00:00+05:30,393.0,WIPRO.NS,-10.87,-2.27,0.25,714.0,375.0,5.32,94.68,50.0,425.0,19.0,19.10,8.0
0,2023-01-02 00:00:00+05:30,435.0,GODREJIND.NS,-4.17,-0.88,0.23,644.0,405.0,12.57,87.43,44.0,638.0,39.5,18.90,47.0
2,2023-01-02 00:00:00+05:30,577.0,BERGEPAINT.NS,-8.37,-0.63,-0.50,787.0,558.0,8.43,91.57,35.0,782.0,72.9,59.30,36.0
38,2023-01-02 00:00:00+05:30,4322.0,LTIM.NS,-7.65,-1.88,-0.70,7506.0,3796.0,14.18,85.82,40.0,4675.0,26.6,50.00,8.0
